In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from category_encoders import BaseNEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn import model_selection
import scipy.stats as stats
from sklearn.feature_selection import SelectFromModel

In [68]:
train = pd.read_csv('../../Feature_Engineering/data/other-cleaned_train.csv')
test = pd.read_csv('../../Feature_Engineering/data/other-cleaned_test.csv')

In [69]:
train.drop(columns = ['Unnamed: 0'], inplace = True)
test.drop(columns = ['Unnamed: 0'], inplace = True)

In [70]:
X_train = train.copy()
X_test = test.copy()

In [71]:
categ_columns = train.drop(columns = ["Opportunity_ID","ID", "Pricing, Delivery_Terms_Quote_Appr",\
                                    "Bureaucratic_Code_0_Approval","Bureaucratic_Code_0_Approved",\
                                    "Submitted_for_Approval","ASP","ASP_(converted)","TRF","Total_Amount",\
                                    "Total_Taxable_Amount","diferencia_en_dias","Last_Modified_DOY","Last_Modified_Year",\
                                    "Opportunity_Created_DOY","Opportunity_Created_Year","Quote_Expiry_DOY",\
                                     "Quote_Expiry_Year","Planned_Delivery_Start_DOY","Planned_Delivery_Start_Year",\
                                    "Planned_Delivery_End_DOY","Planned_Delivery_End_Year",\
                                    "Target"]).columns
for column in categ_columns:
    encoder = BaseNEncoder(base = 5)
    encoder.fit(train[column], train['Target'])
    feature_encoded = encoder.transform(train[column])
    X_train = X_train.join(feature_encoded.add_suffix('_baseN'))
    X_train.drop(columns=[column], inplace = True)

C:\Users\ferna\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\ferna\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\ferna\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\ferna\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\ferna\Anaconda3\lib\site-packages\category_encoders

In [72]:
categ_columns = test.drop(columns = ["Opportunity_ID","ID", "Pricing, Delivery_Terms_Quote_Appr",\
                                    "Bureaucratic_Code_0_Approval","Bureaucratic_Code_0_Approved",\
                                    "Submitted_for_Approval","ASP","ASP_(converted)","TRF","Total_Amount",\
                                    "Total_Taxable_Amount","diferencia_en_dias","Last_Modified_DOY","Last_Modified_Year",\
                                    "Opportunity_Created_DOY","Opportunity_Created_Year","Quote_Expiry_DOY",\
                                     "Quote_Expiry_Year","Planned_Delivery_Start_DOY","Planned_Delivery_Start_Year",\
                                    "Planned_Delivery_End_DOY","Planned_Delivery_End_Year"]).columns
for column in categ_columns:
    encoder = BaseNEncoder(base = 5)
    encoder.fit(train[column], train['Target'])
    feature_encoded = encoder.transform(test[column])
    X_test = X_test.join(feature_encoded.add_suffix('_baseN'))
    X_test.drop(columns=[column], inplace = True)

C:\Users\ferna\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\ferna\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\ferna\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\ferna\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\ferna\Anaconda3\lib\site-packages\category_encoders

In [73]:
X_train["Total_Amount"] = pd.to_numeric(X_train["Total_Amount"],errors='coerce').fillna(X_train["Total_Amount"].mean())
X_train["Opportunity_Created_Year"] = pd.to_numeric(X_train["Opportunity_Created_Year"],errors='coerce').fillna(0)
X_train["Quote_Expiry_DOY"] = pd.to_numeric(X_train["Quote_Expiry_DOY"],errors='coerce').fillna(0)
X_train["Quote_Expiry_Year"] = pd.to_numeric(X_train["Quote_Expiry_Year"],errors='coerce').fillna(0)
X_train["Planned_Delivery_End_DOY"] = pd.to_numeric(X_train["Planned_Delivery_End_DOY"],errors='coerce').fillna(0)
X_train["Planned_Delivery_End_Year"] = pd.to_numeric(X_train["Planned_Delivery_End_Year"],errors='coerce').fillna(0)

X_train = X_train.drop(columns = 'Target')

In [74]:
X_test["Total_Amount"] = pd.to_numeric(X_test["Total_Amount"],errors='coerce').fillna(test["Total_Amount"].mean())
X_test["Opportunity_Created_Year"] = pd.to_numeric(X_test["Opportunity_Created_Year"],errors='coerce').fillna(0)
X_test["Quote_Expiry_DOY"] = pd.to_numeric(X_test["Quote_Expiry_DOY"],errors='coerce').fillna(0)
X_test["Quote_Expiry_Year"] = pd.to_numeric(X_test["Quote_Expiry_Year"],errors='coerce').fillna(0)
X_test["Planned_Delivery_End_DOY"] = pd.to_numeric(X_test["Planned_Delivery_End_DOY"],errors='coerce').fillna(0)
X_test["Planned_Delivery_End_Year"] = pd.to_numeric(X_test["Planned_Delivery_End_Year"],errors='coerce').fillna(0)

## Model: LGBM

In [75]:
def cross_val(model, x_train, y_train):
    score_cross_val = model_selection.cross_val_score(model, x_train, y_train, cv=5)
    print(score_cross_val.mean())

In [76]:
params = {
    'num_leaves':stats.randint(90,5000),
    'min_data_in_leaf':stats.randint(100,1000),
    'max_bin':stats.randint(70,300),
    'learning_rate':stats.uniform(0.01,1-0.01),
    'n_estimators':stats.randint(150,500)
}

def lgbm(x_train, y_train, x_validation, y_validation):
    lgbm_classifier = LGBMClassifier()
    lgbm_gs = RandomizedSearchCV(lgbm_classifier, params, cv=2,verbose=1,n_iter=150)
    lgbm_gs.fit(x_train, y_train)
    lgbm_best = lgbm_gs.best_estimator_
    print(lgbm_gs.best_params_)
    print('lgbm: {}'.format(lgbm_best.score(x_validation, y_validation)))
    return lgbm_best

In [77]:
def test_model(model, x_test, y_test):
    predictions = model.predict_proba(x_test)[:,1]
    logloss = log_loss(y_test, predictions)
    accuracy = accuracy_score(y_test, predictions.round())
    print("Accuracy: %.2f%%, Logloss: %.2f" % (accuracy*100.0, logloss))

In [78]:
def best_features(model,train):
    importance = model.feature_importances_
    result = pd.DataFrame([train.columns,importance]).transpose()
    result.columns = ["Feature","Importance"]
    return result.sort_values(by='Importance', ascending=False).head(15)["Feature"]
    
    
def plot_features(model,train):
    fig = plt.gcf()
    fig.set_size_inches(350, 350)
    selection = SelectFromModel(model, threshold=0.040, prefit=True)
    selected_dataset = selection.transform(train)
    model.plot_importance(booster=model)

    plt.rcParams["figure.figsize"] = (40,20)
    plt.xlabel("\nFeature importance", fontsize=40)
    plt.ylabel("Features", fontsize=35)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.show()

In [79]:
y = train.Target
x_train, x_validation, y_train, y_validation = train_test_split(X_train, y, test_size=0.3, stratify=y)

In [81]:
x_validation.columns

Index(['ID', 'Pricing, Delivery_Terms_Quote_Appr',
       'Bureaucratic_Code_0_Approval', 'Bureaucratic_Code_0_Approved',
       'Submitted_for_Approval', 'Opportunity_ID', 'ASP', 'ASP_(converted)',
       'TRF', 'Total_Amount',
       ...
       'Product_Family_Name_0_baseN', 'Product_Family_Name_1_baseN',
       'Product_Family_Name_2_baseN', 'Product_Family_Name_3_baseN',
       'Product_Family_Name_4_baseN', 'ASP_Higher_Mean_baseN',
       'TRF_Higher_Mean_baseN', 'Same_Owner_baseN',
       'Last_Modified_By_Account_Owner_baseN',
       'Last_Modified_By_Opportunity_Owner_baseN'],
      dtype='object', length=116)

In [82]:
lgbm_model = lgbm(x_train, y_train, x_validation, y_validation)
test_model(lgbm_model,x_validation,y_validation)
cross_val(lgbm_model, x_train, y_train)

Fitting 2 folds for each of 150 candidates, totalling 300 fits


C:\Users\ferna\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\ferna\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ferna\Anaconda3\lib\site-packages\lightgbm\sklearn.py", line 857, in fit
    callbacks=callbacks, init_model=init_model)
  File "C:\Users\ferna\Anaconda3\lib\site-packages\lightgbm\sklearn.py", line 617, in fit
    callbacks=callbacks, init_model=init_model)
  File "C:\Users\ferna\Anaconda3\lib\site-packages\lightgbm\engine.py", line 231, in train
    booster = Booster(params=params, train_set=train_set)
  File "C:\Users\ferna\Anaconda3\lib\site-packages\lightgbm\basic.py", line 2053, in __init__
    train_set.construct()
  File "C:\Users\ferna\Anaconda3\

LightGBMError: Do not support special JSON characters in feature name.

In [ ]:
x_best_train

In [ ]:
best_features = best_features(lgbm_model,X_train)

In [ ]:
y_pred = lgbm_model.predict_proba(X_test)[:,1]
submission_lgbm = pd.DataFrame(data={'Opportunity_ID':X_test['Opportunity_ID'], 'Target': y_pred})
submission_lgbm = submission_lgbm.groupby("Opportunity_ID").agg({"Target":"mean"}).reset_index()
submission_lgbm.to_csv('../submits/lgbm_with_base_n_encoding.csv', index=False)

### lgbm with best features

In [ ]:
X_train_best_features = X_train.loc[:,best_features]
X_test_best_features = X_test.loc[:,best_features]

In [ ]:
x_best_train, x_best_validation, y_best_train, y_best_validation = train_test_split(X_train_best_features, y, test_size=0.3, stratify=y)

In [ ]:
lgbm_model_2 = lgbm(x_best_train, y_best_train, x_best_validation, y_best_validation)
test_model(lgbm_model_2,x_best_validation,y_best_validation)
cross_val(lgbm_model_2, x_best_train, y_best_train)

In [ ]:
y_pred_2 = lgbm_model_2.predict_proba(X_test_best_features)[:,1]
submission_lgbm_2 = pd.DataFrame(data={'Opportunity_ID':X_test_best_features['Opportunity_ID'], 'Target': y_pred_2})
submission_lgbm_2 = submission_lgbm_2.groupby("Opportunity_ID").agg({"Target":"mean"}).reset_index()
submission_lgbm_2.to_csv('../submits/lgbm_best_features_with_base_n_encoding.csv', index=False)